In [19]:
import cv2
import numpy as np
import os
import mediapipe as mp
import pandas as pd
from sklearn.preprocessing import normalize

In [2]:
if not os.path.exists('./cos_sample_image'):
    os.mkdir('./cos_sample_image')

In [3]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# train skeleton extract

In [4]:
# create dataframe base
lv1 = [x.name for x in list(mp_pose.PoseLandmark)]
lv2 = ['x', 'y', 'z', 'v']
columns = pd.MultiIndex.from_product([lv1, lv2]).insert(0, ('playtime', 'sec'))
df = pd.DataFrame(columns=columns)

cap = cv2.VideoCapture("./cos_sample_train.mp4")

# video info
fps = cap.get(cv2.CAP_PROP_FPS)
delay = int(1000/fps)
print("height :", cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print("width :", cap.get(cv2.CAP_PROP_FRAME_WIDTH))
print("fps :", fps)

#cnt = -1

# make mediapipe pose model
with mp_pose.Pose(static_image_mode=False,
                    model_complexity=1,
                    smooth_landmarks=True,
                    enable_segmentation=False,
                    smooth_segmentation=True,
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5,) as pose:
    
    while cap.isOpened():
        
        ret, frame = cap.read()
        
        if not ret:
            break
        
        #playtime = round(cap.get(cv2.CAP_PROP_POS_MSEC)/1000, 2)
        
        # make skeleton image
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img.flags.writeable = False
        results = pose.process(img)
        img.flags.writeable = True
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark
            #print(landmarks)
        except:
            pass

        #mp_drawing.draw_landmarks(img, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        #cv2.imshow("test", img)
        
        #cnt += 1
        
        # save image & skeleton data
        #if cv2.waitKey(delay) & 0xFF == ord('c'):
        #if cnt % 30 == 0:
        cv2.imwrite(f"./cos_sample_image/{cap.get(cv2.CAP_PROP_POS_FRAMES)}_train.jpg", img) #frame : 골격x / img : 골격o
        xyzv = np.array([[res.x, res.y, res.z, res.visibility]for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
        xyzv = np.insert(xyzv, 0, cap.get(cv2.CAP_PROP_POS_FRAMES))
        df = pd.concat([df, pd.DataFrame([xyzv], columns=columns)], ignore_index=True)
        #continue
            
        if cv2.waitKey(delay) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()
    df.to_csv("./cos_sample_image/cos_sample_train_skeleton.csv", index=False) # save skeleton data

height : 720.0
width : 640.0
fps : 29.97002997002997


# test skeleton extract

In [5]:
# create dataframe base
lv1 = [x.name for x in list(mp_pose.PoseLandmark)]
lv2 = ['x', 'y', 'z', 'v']
columns = pd.MultiIndex.from_product([lv1, lv2]).insert(0, ('playtime', 'sec'))
df = pd.DataFrame(columns=columns)

cap = cv2.VideoCapture("./cos_sample_test.mp4")

# video info
fps = cap.get(cv2.CAP_PROP_FPS)
delay = int(1000/fps)
print("height :", cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print("width :", cap.get(cv2.CAP_PROP_FRAME_WIDTH))
print("fps :", fps)

#cnt = -1

# make mediapipe pose model
with mp_pose.Pose(static_image_mode=False,
                    model_complexity=1,
                    smooth_landmarks=True,
                    enable_segmentation=False,
                    smooth_segmentation=True,
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5,) as pose:
    
    while cap.isOpened():
        
        ret, frame = cap.read()
        
        if not ret:
            break
        
        #playtime = round(cap.get(cv2.CAP_PROP_POS_MSEC)/1000, 2)
        
        # make skeleton image
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img.flags.writeable = False
        results = pose.process(img)
        img.flags.writeable = True
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark
            #print(landmarks)
        except:
            pass

        #mp_drawing.draw_landmarks(img, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        #cv2.imshow("test", img)
        
        #cnt += 1
        
        # save image & skeleton data
        #if cv2.waitKey(delay) & 0xFF == ord('c'):
        #if cnt % 30 == 0:
        cv2.imwrite(f"./cos_sample_image/{cap.get(cv2.CAP_PROP_POS_FRAMES)}_test.jpg", img) #frame : 골격x / img : 골격o
        xyzv = np.array([[res.x, res.y, res.z, res.visibility]for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
        xyzv = np.insert(xyzv, 0, cap.get(cv2.CAP_PROP_POS_FRAMES))
        df = pd.concat([df, pd.DataFrame([xyzv], columns=columns)], ignore_index=True)
        #continue
            
        if cv2.waitKey(delay) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()
    df.to_csv("./cos_sample_image/cos_sample_test_skeleton.csv", index=False) # save skeleton data

height : 720.0
width : 640.0
fps : 29.97002997002997


# read train, test data

In [6]:
train = pd.read_csv("./cos_sample_image/cos_sample_train_skeleton.csv", header=[0,1])

In [7]:
test = pd.read_csv("./cos_sample_image/cos_sample_test_skeleton.csv", header=[0,1])

In [8]:
train.head()

playtime      NOSE                               LEFT_EYE_INNER            \
       sec         x         y         z         v              x         y   
0      1.0  0.530722  0.288515 -0.418072  0.999998       0.536021  0.274355   
1      2.0  0.531541  0.288517 -0.418771  0.999998       0.536534  0.274353   
2      3.0  0.531779  0.286608 -0.394940  0.999995       0.536731  0.272949   
3      4.0  0.532713  0.285407 -0.400416  0.999992       0.537325  0.271886   
4      5.0  0.533324  0.283762 -0.397750  0.999990       0.537871  0.270842   

                       LEFT_EYE  ... RIGHT_HEEL           LEFT_FOOT_INDEX  \
          z         v         x  ...          z         v               x   
0 -0.394486  0.999993  0.540506  ...   0.313760  0.885387        0.563856   
1 -0.394490  0.999992  0.540921  ...   0.398194  0.887266        0.564073   
2 -0.366800  0.999977  0.541114  ...   0.394549  0.888964        0.564106   
3 -0.371925  0.999959  0.541724  ...   0.379305  0.889909        0.564040   
4 -0.367384  0.999936  0.542315  ...   0.369387  0.891310        0.563863   

                                RIGHT_FOOT_INDEX                                
          y         z         v                x         y         z         v  
0  0.829309  0.148345  0.990084         0.507596  0.830114  0.134731  0.988936  
1  0.828517  0.252427  0.990203         0.507779  0.830136  0.222445  0.989117  
2  0.827820  0.262247  0.990189         0.507773  0.830136  0.215165  0.989273  
3  0.827544  0.244534  0.989954         0.507768  0.830144  0.196385  0.989313  
4  0.827439  0.222800  0.989689         0.507735  0.830162  0.185588  0.989340  

[5 rows x 133 columns]

In [9]:
test.head()

playtime      NOSE                               LEFT_EYE_INNER            \
       sec         x         y         z         v              x         y   
0      1.0  0.517440  0.191790 -0.356848  0.999980       0.525520  0.180241   
1      2.0  0.515312  0.191814 -0.262179  0.999981       0.524741  0.180255   
2      3.0  0.511461  0.192231 -0.286979  0.999977       0.522251  0.180483   
3      4.0  0.509108  0.193643 -0.367301  0.999971       0.518850  0.181364   
4      5.0  0.508706  0.197692 -0.395361  0.999965       0.517989  0.185085   

                       LEFT_EYE  ... RIGHT_HEEL           LEFT_FOOT_INDEX  \
          z         v         x  ...          z         v               x   
0 -0.308863  0.999951  0.531481  ...   0.344362  0.927483        0.588020   
1 -0.212149  0.999953  0.530590  ...   0.303045  0.927459        0.587197   
2 -0.239952  0.999946  0.528537  ...   0.297698  0.927382        0.587022   
3 -0.323281  0.999933  0.525608  ...   0.299199  0.925959        0.585917   
4 -0.350740  0.999922  0.524685  ...   0.288224  0.925201        0.585518   

                                RIGHT_FOOT_INDEX                                
          y         z         v                x         y         z         v  
0  0.871516  0.022288  0.973722         0.435271  0.850148  0.137319  0.976482  
1  0.869676  0.032775  0.974414         0.434729  0.851558  0.092860  0.977257  
2  0.865889  0.038175  0.974965         0.432286  0.852560  0.086364  0.977879  
3  0.860315  0.043411  0.975676         0.433114  0.851815  0.085901  0.978507  
4  0.854297  0.046811  0.976267         0.433030  0.851337  0.069414  0.979128  

[5 rows x 133 columns]

# extract x, y in raw data

In [10]:
train_xy = train[[(a,b) for a, b in train.columns if b in ['x','y']]]

In [11]:
test_xy = test[[(a,b )for a, b in test.columns if b in ['x', 'y']]]

In [12]:
train_xy.head()

NOSE           LEFT_EYE_INNER            LEFT_EYE            \
          x         y              x         y         x         y   
0  0.530722  0.288515       0.536021  0.274355  0.540506  0.274250   
1  0.531541  0.288517       0.536534  0.274353  0.540921  0.274250   
2  0.531779  0.286608       0.536731  0.272949  0.541114  0.272977   
3  0.532713  0.285407       0.537325  0.271886  0.541724  0.271931   
4  0.533324  0.283762       0.537871  0.270842  0.542315  0.270960   

  LEFT_EYE_OUTER           RIGHT_EYE_INNER            ... RIGHT_ANKLE  \
               x         y               x         y  ...           x   
0       0.545191  0.274156        0.518991  0.275286  ...    0.511483   
1       0.545709  0.274160        0.519665  0.275248  ...    0.511481   
2       0.545953  0.273080        0.519876  0.273632  ...    0.511337   
3       0.546528  0.272094        0.520972  0.272395  ...    0.511129   
4       0.547047  0.271220        0.521723  0.271190  ...    0.510768   

            LEFT_HEEL           RIGHT_HEEL           LEFT_FOOT_INDEX  \
          y         x         y          x         y               x   
0  0.784886  0.562543  0.793876   0.519781  0.798034        0.563856   
1  0.782006  0.562445  0.789262   0.519233  0.790564        0.564073   
2  0.781147  0.562497  0.787841   0.518558  0.789669        0.564106   
3  0.780958  0.562530  0.787342   0.517924  0.789782        0.564040   
4  0.780843  0.562514  0.787462   0.517047  0.789852        0.563863   

            RIGHT_FOOT_INDEX            
          y                x         y  
0  0.829309         0.507596  0.830114  
1  0.828517         0.507779  0.830136  
2  0.827820         0.507773  0.830136  
3  0.827544         0.507768  0.830144  
4  0.827439         0.507735  0.830162  

[5 rows x 66 columns]

In [13]:
test_xy.head()

NOSE           LEFT_EYE_INNER            LEFT_EYE            \
          x         y              x         y         x         y   
0  0.517440  0.191790       0.525520  0.180241  0.531481  0.180967   
1  0.515312  0.191814       0.524741  0.180255  0.530590  0.180986   
2  0.511461  0.192231       0.522251  0.180483  0.528537  0.181172   
3  0.509108  0.193643       0.518850  0.181364  0.525608  0.181895   
4  0.508706  0.197692       0.517989  0.185085  0.524685  0.185464   

  LEFT_EYE_OUTER           RIGHT_EYE_INNER            ... RIGHT_ANKLE  \
               x         y               x         y  ...           x   
0       0.538088  0.182000        0.506933  0.179594  ...    0.464793   
1       0.536555  0.182032        0.505676  0.179655  ...    0.465125   
2       0.534004  0.182185        0.502326  0.180036  ...    0.465270   
3       0.530920  0.182657        0.498691  0.181288  ...    0.465648   
4       0.529591  0.185836        0.497849  0.185010  ...    0.465786   

            LEFT_HEEL           RIGHT_HEEL           LEFT_FOOT_INDEX  \
          y         x         y          x         y               x   
0  0.796986  0.555255  0.839984   0.470715  0.820219        0.588020   
1  0.798281  0.555530  0.837718   0.471077  0.821383        0.587197   
2  0.798566  0.555894  0.834878   0.471470  0.822034        0.587022   
3  0.798527  0.556102  0.830296   0.472023  0.821977        0.585917   
4  0.799773  0.556113  0.827486   0.472339  0.824214        0.585518   

            RIGHT_FOOT_INDEX            
          y                x         y  
0  0.871516         0.435271  0.850148  
1  0.869676         0.434729  0.851558  
2  0.865889         0.432286  0.852560  
3  0.860315         0.433114  0.851815  
4  0.854297         0.433030  0.851337  

[5 rows x 66 columns]

# flatten data by frame

In [15]:
train_list = train_xy.values

In [16]:
test_list = test_xy.values

In [17]:
train_list

array([[0.53072172, 0.28851521, 0.53602087, ..., 0.82930911, 0.50759625,
        0.83011401],
       [0.53154135, 0.28851709, 0.53653449, ..., 0.82851696, 0.50777948,
        0.83013618],
       [0.53177887, 0.28660843, 0.53673083, ..., 0.82781965, 0.5077728 ,
        0.83013576],
       ...,
       [0.52776146, 0.28853187, 0.53469318, ..., 0.82671815, 0.507568  ,
        0.8295188 ],
       [0.52807343, 0.28867352, 0.53479505, ..., 0.82660395, 0.50701976,
        0.82944769],
       [0.52738917, 0.28881958, 0.53424871, ..., 0.82603091, 0.50701463,
        0.82916361]])

In [18]:
test_list

array([[0.51744032, 0.19179034, 0.52552009, ..., 0.87151581, 0.43527079,
        0.8501476 ],
       [0.51531196, 0.19181417, 0.52474076, ..., 0.86967587, 0.43472892,
        0.85155767],
       [0.5114609 , 0.19223098, 0.52225101, ..., 0.86588943, 0.43228585,
        0.85255998],
       ...,
       [0.49047166, 0.20908014, 0.50032771, ..., 0.84921831, 0.43841267,
        0.84010637],
       [0.49122792, 0.20719719, 0.50102711, ..., 0.85077542, 0.43907666,
        0.84147143],
       [0.49136463, 0.20585416, 0.50134426, ..., 0.85116839, 0.43951377,
        0.84170628]])

# cosine similarity - 함수 정의

In [57]:
def cos_sim(x_array, y_array):
    cossim_list = []
    for x, y in zip(x_array, y_array):
        cossim = np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))
        cossim_list.append(cossim)
    return cossim_list

In [58]:
np.mean(cos_sim(train_list, test_list))

0.9897099269414873

In [59]:
np.mean(cos_sim(train_list, train_list))

1.0

In [60]:
train_norm = normalize(train_list)

In [61]:
test_norm = normalize(test_list)

In [62]:
np.mean(cos_sim(test_norm, train_norm))

0.9897099269414873

In [63]:
np.mean(cos_sim(train_norm, train_norm))

1.0

# cosine similarity - 함수 사용

In [64]:
from sklearn.metrics.pairwise import cosine_similarity

In [65]:
cosine_similarity(train_list, test_list).mean()

0.9907852441030097

In [66]:
cosine_similarity(test_list, test_list).mean()  # 왜 같은 영상을 넣었는데 1이 안나오지..?

0.9954990993964469

# make error data : clap_clap

In [43]:
if not os.path.exists('./error_sample_image'):
    os.mkdir('./error_sample_image')

In [44]:
# create dataframe base
lv1 = [x.name for x in list(mp_pose.PoseLandmark)]
lv2 = ['x', 'y', 'z', 'v']
columns = pd.MultiIndex.from_product([lv1, lv2]).insert(0, ('playtime', 'sec'))
df = pd.DataFrame(columns=columns)

cap = cv2.VideoCapture("./sample/clap_clap.mp4")

# video info
fps = cap.get(cv2.CAP_PROP_FPS)
delay = int(1000/fps)
print("height :", cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print("width :", cap.get(cv2.CAP_PROP_FRAME_WIDTH))
print("fps :", fps)

#cnt = -1

# make mediapipe pose model
with mp_pose.Pose(static_image_mode=False,
                    model_complexity=1,
                    smooth_landmarks=True,
                    enable_segmentation=False,
                    smooth_segmentation=True,
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5,) as pose:
    
    while cap.isOpened():
        
        ret, frame = cap.read()
        
        if not ret:
            break
        
        #playtime = round(cap.get(cv2.CAP_PROP_POS_MSEC)/1000, 2)
        
        # make skeleton image
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img.flags.writeable = False
        results = pose.process(img)
        img.flags.writeable = True
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark
            #print(landmarks)
        except:
            pass

        #mp_drawing.draw_landmarks(img, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        #cv2.imshow("test", img)
        
        #cnt += 1
        
        # save image & skeleton data
        #if cv2.waitKey(delay) & 0xFF == ord('c'):
        #if cnt % 30 == 0:
        cv2.imwrite(f"./error_sample_image/{cap.get(cv2.CAP_PROP_POS_FRAMES)}_clap_clap.jpg", img) #frame : 골격x / img : 골격o
        xyzv = np.array([[res.x, res.y, res.z, res.visibility]for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
        xyzv = np.insert(xyzv, 0, cap.get(cv2.CAP_PROP_POS_FRAMES))
        df = pd.concat([df, pd.DataFrame([xyzv], columns=columns)], ignore_index=True)
        #continue
            
        if cv2.waitKey(delay) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()
    df.to_csv("./error_sample_image/clap_clap_skeleton.csv", index=False) # save skeleton data

height : 720.0
width : 640.0
fps : 29.97002997002997


In [45]:
clap_clap = pd.read_csv("./error_sample_image/clap_clap_skeleton.csv", header=[0,1])

In [46]:
clap_clap_xy = clap_clap[[(a,b) for a, b in clap_clap.columns if b in ['x','y']]]

In [47]:
clap_clap_list = clap_clap_xy.values

In [48]:
clap_clap_list

array([[0.67655832, 0.3142069 , 0.68570638, ..., 0.85780036, 0.6314013 ,
        0.85862172],
       [0.67698324, 0.31429628, 0.68580896, ..., 0.85571772, 0.6313374 ,
        0.85833305],
       [0.67701977, 0.31424239, 0.68568617, ..., 0.85483164, 0.63114011,
        0.85809463],
       ...,
       [0.6538412 , 0.33837292, 0.65690541, ..., 0.85477895, 0.60775387,
        0.85441089],
       [0.65381944, 0.33836651, 0.65686458, ..., 0.85481972, 0.60778356,
        0.85476661],
       [0.65400654, 0.33842838, 0.65704918, ..., 0.85479581, 0.60771698,
        0.85499692]])

# make error data : foot_stamp_clap

In [49]:
# create dataframe base
lv1 = [x.name for x in list(mp_pose.PoseLandmark)]
lv2 = ['x', 'y', 'z', 'v']
columns = pd.MultiIndex.from_product([lv1, lv2]).insert(0, ('playtime', 'sec'))
df = pd.DataFrame(columns=columns)

cap = cv2.VideoCapture("./sample/foot_stamp_clap.mp4")

# video info
fps = cap.get(cv2.CAP_PROP_FPS)
delay = int(1000/fps)
print("height :", cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print("width :", cap.get(cv2.CAP_PROP_FRAME_WIDTH))
print("fps :", fps)

#cnt = -1

# make mediapipe pose model
with mp_pose.Pose(static_image_mode=False,
                    model_complexity=1,
                    smooth_landmarks=True,
                    enable_segmentation=False,
                    smooth_segmentation=True,
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5,) as pose:
    
    while cap.isOpened():
        
        ret, frame = cap.read()
        
        if not ret:
            break
        
        #playtime = round(cap.get(cv2.CAP_PROP_POS_MSEC)/1000, 2)
        
        # make skeleton image
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img.flags.writeable = False
        results = pose.process(img)
        img.flags.writeable = True
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark
            #print(landmarks)
        except:
            pass

        #mp_drawing.draw_landmarks(img, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        #cv2.imshow("test", img)
        
        #cnt += 1
        
        # save image & skeleton data
        #if cv2.waitKey(delay) & 0xFF == ord('c'):
        #if cnt % 30 == 0:
        cv2.imwrite(f"./error_sample_image/{cap.get(cv2.CAP_PROP_POS_FRAMES)}_foot_stamp_clap.jpg", img) #frame : 골격x / img : 골격o
        xyzv = np.array([[res.x, res.y, res.z, res.visibility]for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
        xyzv = np.insert(xyzv, 0, cap.get(cv2.CAP_PROP_POS_FRAMES))
        df = pd.concat([df, pd.DataFrame([xyzv], columns=columns)], ignore_index=True)
        #continue
            
        if cv2.waitKey(delay) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()
    df.to_csv("./error_sample_image/foot_stamp_clap_skeleton.csv", index=False) # save skeleton data

height : 720.0
width : 640.0
fps : 29.97002997002997


In [50]:
foot_stamp_clap = pd.read_csv("./error_sample_image/foot_stamp_clap_skeleton.csv", header=[0,1])

In [51]:
foot_stamp_clap_xy = foot_stamp_clap[[(a,b) for a, b in foot_stamp_clap.columns if b in ['x','y']]]

In [52]:
foot_stamp_clap_list = foot_stamp_clap_xy.values

In [53]:
foot_stamp_clap_list

array([[0.65600443, 0.33856195, 0.6607486 , ..., 0.85086447, 0.60138524,
        0.85469222],
       [0.65531468, 0.33856973, 0.66015792, ..., 0.85343307, 0.60300469,
        0.85506779],
       [0.65675384, 0.33668578, 0.66090906, ..., 0.85445499, 0.60415351,
        0.85524213],
       ...,
       [0.65450698, 0.34808812, 0.65619451, ..., 0.86594146, 0.60691935,
        0.8552568 ],
       [0.65451008, 0.34964326, 0.65621746, ..., 0.86395639, 0.60690224,
        0.8552314 ],
       [0.65433633, 0.35043493, 0.65621436, ..., 0.86334312, 0.60673183,
        0.85546768]])

# cosine similarity - 함수 정의

In [67]:
np.mean(cos_sim(train_list, clap_clap_list))

0.9885622356329111

In [68]:
np.mean(cos_sim(train_list, foot_stamp_clap_list))

0.984568747733488

# cosine similarity - 함수 사용

In [69]:
cosine_similarity(train_list, clap_clap_list).mean()

0.9880033618617248

In [71]:
cosine_similarity(train_list, foot_stamp_clap_list).mean()

0.9861463805928109

- 코사인 유사도가 너무 높게 나옴
- 동일한 영상 : 1
- 동일한 동작의 다른 영상 : 0.99
- 다른 동작의 다른 영상 : 0.988